##  Insurance Data Analysis

### Exploratory Data Analysis


In [25]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os, sys

sys.path.append('../scripts') 

In [26]:
# Import load_data function from scripts
from load_data import load_data

data = load_data('../data/raw/MachineLearningRating_v3.zip', filename='MachineLearningRating_v3.txt')

In [ ]:
data.tail()

In [ ]:
data.shape

### Data Cleaning

In [ ]:
# Import the Class to data processing

from data_processing import DataProcessing
# Create instance of the class
data_processing = DataProcessing(data)

# Summary of Missing data
missing_summary = data_processing.missing_data_summary()

# Display results
missing_summary

In [30]:
cols_to_drop = ['NumberOfVehiclesInFleet', 
                'CrossBorder', 
                'CustomValueEstimate', 
                'Converted', 'Rebuilt', 
                'WrittenOff']

# Drop these columns
data = data_processing.handle_missing_data('high', cols_to_drop)


In [31]:
# Impute or drop columns with moderate missing data
missing_cols = ['NewVehicle', 'Bank', 'AccountType']
data = data_processing.handle_missing_data('moderate', missing_cols)

In [32]:
# Handle low missing data (standard imputation)
missing_cols = ['Gender', 'MaritalStatus', 'Cylinders', 'cubiccapacity', 
                'kilowatts', 'NumberOfDoors', 'VehicleIntroDate', 'Model', 
                'make', 'VehicleType', 'mmcode', 'bodytype', 'CapitalOutstanding']

data = data_processing.handle_missing_data('low', missing_cols)


In [ ]:
data.duplicated().sum()

In [ ]:
# Statistic summary of numerical features
num_cols = ['SumInsured', 'CalculatedPremiumPerTerm', 'TotalPremium', 'TotalClaims']
display(data[num_cols].describe())

In [ ]:
# Check the Data types of each column

data_types = data.dtypes

data_types.reset_index(name='data type')

In [36]:
# Converts the VehicleIntroDate column from object to datetime format.

data['VehicleIntroDate'] = pd.to_datetime(data['VehicleIntroDate'], format='%d/%m/%Y', errors='coerce')  # Convert with error handling

### Univarate Analysis

In [37]:
# Import visualizer
from data_visualization import DataVisualizer
# Create instance of the class
vis = DataVisualizer(data)

# Define numerical columns
numerical_cols = [
    'SumInsured', 'CalculatedPremiumPerTerm', 
    'TotalPremium', 'TotalClaims'
]
# Define categorical columns
categorical_cols = [
    'LegalType', 'Bank', 'AccountType', 'MaritalStatus', 
    'Gender', 'Province', 'VehicleType', 
    'AlarmImmobiliser', 'TrackingDevice', 
    'Product'
]


**Insights from Univariate Analysis**

**Numerical Variables:**

- **Sum Insured:** The distribution is right-skewed, with most policies having lower sums insured and a few with very high amounts. This indicates a diverse range of policy values, with clusters around specific sums suggesting potential policy tiers.

- **Calculated Premium Per Term:** This variable is also right-skewed, with a peak around a specific amount and variability in premiums. The presence of significantly higher premiums reflects pricing differences among policies.

- **Total Premium:** The distribution is right-skewed, with most policies showing lower total premiums and a peak around zero. High total premiums may be due to refunds or high-risk factors.

Overall, these distributions reveal a wide range of policy values and premiums, with identifiable clusters and outliers that can aid in risk assessment and pricing strategies.

**Categorical Variables:**

- **Legal Type:** Most policies are issued to individuals, indicating a focus on personal insurance. Private companies and close corporations have fewer policies, with public companies, partnerships, and sole proprietorships being even less common.

- **Marital Status:** A significant portion of data has "Not specified" for marital status, suggesting incomplete information. Single and married individuals are less represented, highlighting potential issues with data collection or privacy concerns.

- **Province:** Gauteng has the highest number of policies, with Western Cape and KwaZulu-Natal showing moderate numbers. Other provinces have fewer policies, indicating a concentration of market presence in specific regions.

- **Vehicle Type:** Passenger vehicles dominate, with fewer policies for commercial vehicle types (medium, heavy, light, and bus). This underscores the company's focus on passenger vehicles.

- **Gender:** A substantial portion of data has "Not specified" for gender, indicating missing information. Male and female categories are present but less common, suggesting potential gaps in data collection or reporting.

**Bivariate or Multivariate Analysis:**

- **Correlations and Associations:** Explore relationships between the monthly changes `TotalPremium` and `TotalClaims` as a function of ZipCode, using scatter plots and correlation matrices.


### Data Comparison

In [ ]:
# Cover type frequence distributions accross the regions

cover_type_counts = data['CoverType'].value_counts()

# Create a bar chart with a color palette
plt.figure(figsize=(12, 4))
sns.barplot(x=cover_type_counts.index, y=cover_type_counts.values, palette='viridis')
plt.title('Cover Type Frequencies')
plt.xlabel('Cover Type')
plt.ylabel('Count')
plt.xticks(rotation=90)  # Rotate labels to the bottom
plt.show()

In [39]:
# Define the most common cover types
common_cover_types = [
    'Own Damage', 'Passenger Liability', 'Windscreen', 'Third Party', 
    'Keys and Alarms', 'Signage and Vehicle Wraps', 'Emergency Charges', 
    'Cleaning and Removal of Accidental Debris'
]

# Ensure the output directory exists before saving the plot
os.makedirs('Data/visualizations', exist_ok=True)

# Plot the distribution accross regions
vis.plot_geographical_trends(common_cover_types)


In [40]:
data.to_csv('../data/cleaned/MachineLearningRating_v3.txt', sep='|', index=False)

#### 1. Loss Ratio By Province

In [ ]:
# Compute Loss Ratio
data["LossRatio"] = data["TotalClaims"] / data["TotalPremium"]

# Plot
plt.figure(figsize=(12,6))
sns.boxplot(data=data, x="Province", y="LossRatio")
plt.xticks(rotation=45)
plt.title("Loss Ratio Distribution by Province", fontsize=14)
plt.ylabel("Loss Ratio")
plt.xlabel("Province")
plt.tight_layout()
plt.show()

#### Total Claims by Vehicle Make (Top 10)

In [ ]:
# Group and sort by claims
top_makes = data.groupby("make")["TotalClaims"].sum().sort_values(ascending=False).head(10)

# Plot
plt.figure(figsize=(10,6))
sns.barplot(x=top_makes.values, y=top_makes.index, palette="magma")
plt.title("Top 10 Vehicle Makes by Total Claims", fontsize=14)
plt.xlabel("Total Claims")
plt.ylabel("Vehicle Make")
plt.tight_layout()
plt.show()

### Loss Ratio by Gender

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(data=data, x="Gender", y="LossRatio", palette="Set2")
plt.title("Loss Ratio by Gender", fontsize=14)
plt.xlabel("Gender")
plt.ylabel("Loss Ratio")
plt.tight_layout()
plt.show()